In [0]:
# Load tables from the catalog
#crossborder = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw__v2.crossborder_flows_all")
#generation = spark.table("workspace.schema_capstone.generation_clean")
#load_actual = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw__v2.load_actual")
#weather = spark.table("curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw__v2.weather_hourly")
all = spark.table("workspace.default.electricity_and_weather_europe")

In [0]:
#create unified df
#df = load_actual
#df = df.join(generation, on=["index", "country"], how="outer")
#df = df.join(capacity, on=["index", "country"], how="outer")

In [0]:
#just explore tables separately
#display(crossborder.limit(15))

In [0]:
all = all.toPandas()
all.head()

In [0]:
crossborder.describe()

In [0]:
all.info()

In [0]:
from pyspark.sql import functions as F
from functools import reduce

df_spark = all  # your Spark DataFrame

# 1. Identify all aggregated generation columns
gen_cols = [c for c in df_spark.columns if c.endswith("__Actual_Aggregated")]

"""

columns_to_average = ["ssrd", "wind_speed", "temperature_c"]
agg_exprs = [F.mean(c).alias(f"mean_{c}") for c in columns_to_average]
country_time_mean = weather_with_country.groupBy("country", "timestamp").agg(*agg_exprs)

country_time_mean = country_time_mean.orderBy("timestamp")

display(country_time_mean)


     
"""

# 2. Compute total generation using reduce()
df_spark = df_spark.withColumn(
    "total_generation",
    reduce(lambda a, b: a + b, (F.col(c) for c in gen_cols))
)

# 3. Compute reserve margin
df_spark = df_spark.withColumn(
    "reserve_margin",
    (F.col("total_generation") - F.col("Actual_Load")) / F.col("Actual_Load")
)

# 4. blackout_risk = 1 if reserve margin < 5%
df_spark = df_spark.withColumn(
    "blackout_risk",
    (F.col("reserve_margin") < 0.05).cast("int")
)

# Optional: force negative margins to be blackout events
df_spark = df_spark.withColumn(
    "blackout_risk",
    F.when(F.col("reserve_margin") < 0, 1).otherwise(F.col("blackout_risk"))
)

display(df_spark.select("total_generation", "reserve_margin", "blackout_risk"))


In [0]:
import seaborn as sns
sns.heatmap(crossborder.isnull(), cbar=False, cmap='viridis')

In [0]:
generation = generation.toPandas()
generation.head()

In [0]:
generation.describe()

In [0]:
generation.info()

In [0]:
import seaborn as sns
sns.heatmap(generation.isnull(), cbar=False, cmap='viridis')


In [0]:
load_actual = load_actual.toPandas()
load_actual.head()

In [0]:
#load_actual['month_of_date'] = load_actual['index'].dt.month